In [1]:
import pandas as pd
import numpy as np
import pickle 
import os

In [5]:

# Step 1: Open the file in read-binary mode
try:
    with open('data.pkl', 'rb') as file:
        # Step 2: Load the data from the file
        data = pickle.load(file)
    print(data)
except FileNotFoundError:
    print("The file 'data.pkl' does not exist.")
except pickle.UnpicklingError:
    print("Error occurred while unpickling the file.")


TypeError: Cannot convert numpy.ndarray to numpy.ndarray